In [25]:
import pandas as pd
import random 

from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
    PromptTemplate,
    FewShotPromptTemplate,
)

from langchain.chains import LLMChain

In [5]:
import os
os.environ['OPENAI_API_KEY'] = ''

In [6]:
chat_model=ChatOpenAI()

In [8]:
conversation_chain = ConversationChain(
    llm=chat_model
)

print(conversation_chain.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


# Few-shot learning

In [13]:

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "5+6", "output": "11"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: 5+6
AI: 11


In [15]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wornderous wizard of math."),
        few_shot_prompt,
        ("human", "{input}"),
    ]
)

print(final_prompt.format(input='What is 3+3?'))

System: You are a wornderous wizard of math.
Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: 5+6
AI: 11
Human: What is 3+3?


In [17]:
chain = LLMChain(
    llm=chat_model,
    prompt=final_prompt,
    verbose=True
)

chain.run("What is 3+3?")



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a wornderous wizard of math.
Human: 2+2
AI: 4
Human: 2+3
AI: 5
Human: 5+6
AI: 11
Human: What is 3+3?

> Finished chain.


'3+3 is 6.'

## Memetic Proxy

In [23]:
template = """
System: {reference}
Provide a helpful answer to the following question

Human: {question}

AI:"""

prompt = PromptTemplate.from_template(template)

chain=LLMChain(
    llm=chat_model,
    prompt=prompt,
    verbose=True
)

high_level = "Imagine you are a professor teaching at the PhD level."
lower_level = "Imagine you are a kindergarten teacher."

question = "What is quantum mechanics"

chain.run(
    {
        'question':question,
        'reference':high_level
    }
)



> Entering new LLMChain chain...
Prompt after formatting:

System: Imagine you are a professor teaching at the PhD level.
Provide a helpful answer to the following question

Human: What is quantum mechanics

AI:

> Finished chain.


'Quantum mechanics is a fundamental theory in physics that describes the behavior of matter and energy on very small scales, such as atoms and subatomic particles. It involves principles such as superposition, entanglement, and wave-particle duality, which can seem counterintuitive compared to classical physics. Quantum mechanics has led to groundbreaking discoveries and technological advancements, such as the development of quantum computing and cryptography.'

In [24]:
chain.run(
    {
        'question':question,
        'reference':lower_level
    }
)



> Entering new LLMChain chain...
Prompt after formatting:

System: Imagine you are a kindergarten teacher.
Provide a helpful answer to the following question

Human: What is quantum mechanics

AI:

> Finished chain.


'Quantum mechanics is a branch of physics that studies the behavior of particles at a very small scale, such as atoms and subatomic particles. It describes how these particles interact with each other and with energy in ways that are very different from classical physics. Quantum mechanics has led to many important technological advancements, such as the development of computers and lasers.'

## Chain of Thoughts

In [27]:
data = pd.read_json('data/CoT_collection.json')
datat = data.T.copy()
data_sub = datat.loc[datat['task'] == 'ai2_arithmetic_questions']
data_sub['Question'] = data_sub['source'].apply(lambda row: '\n\n'.join(row.split('\n\n')[1:]))
data_final = data_sub.rename({'target': 'Response', 'rationale': 'Rationale'}, axis=1)[['Question', 'Rationale', 'Response']]
data_final.to_csv('data/CoT_collection.csv', index=False)

C:\Users\Owner\AppData\Local\Temp\ipykernel_16260\1492856589.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sub['Question'] = data_sub['source'].apply(lambda row: '\n\n'.join(row.split('\n\n')[1:]))


In [30]:
file_path = 'data/CoT_collection.csv'

cot_df = pd.read_csv(file_path)

cot_df

,Question,Rationale,Response
0,Blake filled a bucket with 0.8 gallon of water...,Since Blake filled the bucket with 0.8 gallon ...,0.6
1,There is 0.2 cup of oil in Scarlett's measurin...,0.2 + 0.3 = 0.5,0.5
2,Craig walked 0.2 mile from school to David's h...,Craig walked 0.2 mile from school to David's h...,0.9
3,Hannah's Vegetarian Restaurant bought 0.3 poun...,Hannah's Vegetarian Restaurant bought 0.3 poun...,0.9
4,Alyssa bought some toys. She bought a football...,Alyssa spent $5.71 on football and $6.59 on ma...,12.3
...,...,...,...
575,"Ella owns 2 dogs. Each day, 1 dog eats 0.175 s...","Ella owns 2 dogs. Each day, 1 dog eats 0.175 s...",0.3
576,"Mike found 6 seashells and 4 starfish, but 4 o...",The problem is: 6 + 4 - 4 / 2.0 = 10.0 - 2.0 =...,2.0
577,Mary loves eating fruits. Mary paid $11.08 for...,Mary loves eating fruits. Mary paid $11.08 for...,34.7
578,Joan has 40 blue balloons Melanie has 41 blue ...,Joan has 40 blue balloons Melanie has 41 blue ...,81.0


In [31]:
list(cot_df.T.to_dict().values())[0]

{'Question': 'Blake filled a bucket with 0.8 gallon of water. Later, he poured out 0.2 gallon of the water. How much water is in the bucket?',
 'Rationale': 'Since Blake filled the bucket with 0.8 gallon of water, and poured out 0.2 gallon of it, the current amount of water left in the bucket is 0.8 - 0.2 = 0.6 gallon',
 'Response': 0.6}

In [32]:
random.sample(list(cot_df.T.to_dict().values()), 10)

[{'Question': "Joan joined her school's band. She bought a trumpet for $149.18, amusic tool for $9.98, and a song book which was $4.14. Joan found$8.65 in her pocket. How much did Joan spend at the music store?",
  'Rationale': "Joan spent $149.18 on a trumpet, $9.98 on a music tool, and $4.14 on a song book. Finding extra money in her pocket doesn't matter with Joan's purchase. (Total cost of Joan's purchase) = 149.18 + 9.98 + 4.14 = 163.3",
  'Response': 163.3},
 {'Question': 'Ella owns 2 dogs. Each day, 1 dog eats 0.175 scoop of dog food andthe other dog eats 0.125 scoop. Together, how much dog food dothe 2 dogs eat each day?',
  'Rationale': 'Ella owns 2 dogs. Each day, 1 dog eats 0.175 scoop of dog food and the other dog eats 0.125 scoop. So, together, the 2 dogs eat (2 x 0.175) + (2 x 0.125) scoops of dog food each day = (0.35) + (0.25) = 0.3 scoops',
  'Response': 0.3},
 {'Question': "Keith spent $6.5 on a rabbit toy, $5.79 on pet food, and a cagecost him $12.51. He found a doll

In [33]:
examples = random.sample(list(cot_df.T.to_dict().values()), 10)

In [35]:
template ="""
Question: {Question}
Rationale: {Rationale}
Response: {Response}
"""

example_prompt = PromptTemplate(
    input_variables=["Question","Rationale","Response"],
    template=template
)

system_prompt="""
You are provided with an arithmetic question.
Your task is to compute the solution using the given arithmetic operations.
The only arithmetic operatos needed to answer the question are '+' (addition) and '-' (substraction).
The answer should be correct to one decimal place.
"""

COT_promt = FewShotPromptTemplate(
    prefix=system_prompt,
    example_prompt=example_prompt,
    examples=examples,
    suffix="Question: {input}",
    input_variables=['input']
)

print(COT_promt.format(input='[EXAMPLE QUESTION]'))


You are provided with an arithmetic question.
Your task is to compute the solution using the given arithmetic operations.
The only arithmetic operatos needed to answer the question are '+' (addition) and '-' (substraction).
The answer should be correct to one decimal place.



Question: For his car, Mike spent $118.57 on speakers and $106.33 on newtires. Mike wanted 3 CD's for $4.58 but decided not to. In total,how much did Mike spend on car parts?
Rationale: Mike spent $118.57 on speakers and $106.33 on new tires. If Mike wanted 3 CD's for $4.58 each he would have spent 0 since he decided not to buy them. (Total cost of car parts) = 118.57 + 106.33 + 0 = 224.90
Response: 224.9



Question: At Lindsey's Vacation Wear, 0.37 the garments are bikinis and 0.23 are trunks. What fraction of the garments are either bikinis or trunks?
Rationale: The fraction of garments that are bikinis is 0.37 and the fraction of garments that are trunks is 0.23, so the total fraction of garments that are ei

In [37]:
chain = LLMChain(
    llm=chat_model,
    prompt=COT_promt,
    verbose=True
)

question ="""
Mandy made an apple pie. She used 0.6 of cinnamon and 0.5 tablespoon of nutmeg.
How much more cinnamon than nutmeg did Mandy use?
"""

chain.run(question)



> Entering new LLMChain chain...
Prompt after formatting:

You are provided with an arithmetic question.
Your task is to compute the solution using the given arithmetic operations.
The only arithmetic operatos needed to answer the question are '+' (addition) and '-' (substraction).
The answer should be correct to one decimal place.



Question: For his car, Mike spent $118.57 on speakers and $106.33 on newtires. Mike wanted 3 CD's for $4.58 but decided not to. In total,how much did Mike spend on car parts?
Rationale: Mike spent $118.57 on speakers and $106.33 on new tires. If Mike wanted 3 CD's for $4.58 each he would have spent 0 since he decided not to buy them. (Total cost of car parts) = 118.57 + 106.33 + 0 = 224.90
Response: 224.9



Question: At Lindsey's Vacation Wear, 0.37 the garments are bikinis and 0.23 are trunks. What fraction of the garments are either bikinis or trunks?
Rationale: The fraction of garments that are bikinis is 0.37 and the fraction of garments that are t

'Rationale: Mandy used 0.6 tablespoon of cinnamon and 0.5 tablespoon of nutmeg. To find out how much more cinnamon than nutmeg she used, we need to subtract the amount of nutmeg from the amount of cinnamon: 0.6 - 0.5 = 0.1\nResponse: 0.1'